In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install torchdata==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.0 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.7.0
    Uninstalling torchdata-0.7.0:
      Successfully uninstalled torchdata-0.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.12.1 which is incompatible.
torchtext 0.16.0 requires torchdata==0.7.0, but you have torchdata 0.4.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but yo

In [3]:
!pip install torchtext==0.13.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.7 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchtext
from torch.nn.utils.rnn import pad_sequence

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

In [5]:
print(torch.__version__)
print(torchtext.__version__)

1.12.1+cu102
0.13.1


## Tokenization

In [6]:
from torchtext import transforms as T

VOCAB_FILE = "https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt"

tokenizer = T.BERTTokenizer(
    vocab_path=VOCAB_FILE, do_lower_case=True, return_tokens=True
)

100%|██████████| 232k/232k [00:00<00:00, 6.61MB/s]


## Vocab (Vectorization)

In [7]:
from torchtext.vocab import GloVe

vocab = GloVe(name="6B", dim=50)

.vector_cache/glove.6B.zip: 862MB [02:41, 5.35MB/s]                           
100%|█████████▉| 399999/400000 [00:07<00:00, 56773.03it/s]


## Dataset

In [8]:
from torchtext import datasets

train_set, test_set = datasets.AG_NEWS("/content/", split=("train", "test"))
train_set, test_set

(ShardingFilterIterDataPipe, ShardingFilterIterDataPipe)

In [19]:
def collate(batch):
    labels = torch.LongTensor([b[0] for b in batch]) - 1
    text = [b[1] for b in batch]
    lines = tokenizer(text)
    vecs = [vocab.get_vecs_by_tokens(line) for line in lines]
    vecs = pad_sequence(vecs)
    return vecs, labels

In [20]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate)
test_loader = DataLoader(test_set, batch_size=64, collate_fn=collate)

In [21]:
x, y = next(iter(train_loader))
x.shape, y

(torch.Size([199, 64, 50]),
 tensor([3, 2, 1, 0, 3, 1, 2, 2, 3, 2, 0, 3, 2, 3, 3, 3, 2, 3, 3, 1, 3, 0, 2, 2,
         3, 2, 2, 2, 2, 2, 3, 3, 1, 1, 1, 0, 3, 1, 1, 3, 0, 2, 0, 0, 0, 3, 0, 2,
         0, 0, 3, 1, 1, 3, 0, 3, 2, 3, 0, 0, 2, 3, 0, 3]))

## Utils

In [12]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Init

In [13]:
num_cls = 4

device = "cuda" if torch.cuda.is_available() else "cpu"

# **Model**

In [48]:
class RNNModel(nn.Module):
    def __init__(
        self, RNN, input_size, hidden_size, num_layers, bidirectional, num_cls
    ):
        super().__init__()
        self.rnn = RNN(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=False,
        )
        self.fc = nn.LazyLinear(
            num_cls
        )  # automatically infers the input size the first time it sees data

    def forward(self, x):
        outputs, _ = self.rnn(
            x
        )  # outputs:hidden states for each time step. _:final hidden state (ignored)
        out = outputs.mean(dim=0)
        y = self.fc(out)

        return y

## Functions

In [37]:
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
    model.train()
    loss_train = AverageMeter()
    correct = 0
    total = 0

    with tqdm(train_loader, unit="batch") as tepoch:
        for inputs, targets in tepoch:
            if epoch is not None:
                tepoch.set_description(f"Epoch {epoch}")
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train.update(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            tepoch.set_postfix(loss=loss_train.avg, accuracy=100 * correct / total)

    accuracy = 100 * correct / total
    return model, loss_train.avg, accuracy

In [38]:
def validation(model, test_loader, loss_fn):
    model.eval()
    loss_valid = AverageMeter()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            loss_valid.update(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    return loss_valid.avg, accuracy

## Train

### Step 1: check forward path

Calculate loss for one batch

In [45]:
model = RNNModel(nn.RNN, 50, 64, 1, False, num_cls).to(device)
loss_fn = nn.CrossEntropyLoss()

x_batch, y_batch = next(iter(train_loader))
outputs = model(x_batch.to(device))
loss = loss_fn(outputs, y_batch.to(device))
print(loss)

tensor(1.3810, grad_fn=<NllLossBackward0>)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:248: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


### Step 2: select best lr

Train all data for one epoch

In [46]:
num_epochs = 1
for lr in [0.1, 0.01, 0.001, 0.0001]:
    print(f"LR={lr}")
    model = RNNModel(nn.RNN, 50, 64, 1, False, num_cls).to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=1e-4, momentum=0.9)
    for epoch in range(num_epochs):
        model, _, _ = train_one_epoch(model, train_loader, loss_fn, optimizer, epoch)
    print()

LR=0.1


Epoch 0: : 1875batch [01:30, 20.62batch/s, accuracy=47.4, loss=1.24]



LR=0.01


Epoch 0: : 1875batch [01:30, 20.61batch/s, accuracy=70, loss=0.817]



LR=0.001


Epoch 0: : 1875batch [01:31, 20.55batch/s, accuracy=47.9, loss=1.29]



LR=0.0001


Epoch 0: : 1875batch [01:30, 20.70batch/s, accuracy=26.7, loss=1.39]

### Step 3: train more epochs

In [50]:
model = RNNModel(nn.RNN, 50, 64, 1, False, num_cls).to(device)

lr = 0.01
wd = 1e-4
optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

loss_train_hist = []
loss_valid_hist = []

acc_train_hist = []
acc_valid_hist = []

best_loss_valid = torch.inf
epoch_counter = 0

In [51]:
num_epochs = 15

for epoch in range(num_epochs):
    # Train
    model, loss_train, acc_train = train_one_epoch(
        model, train_loader, loss_fn, optimizer, epoch
    )
    # Validation
    loss_valid, acc_valid = validation(model, test_loader, loss_fn)

    loss_train_hist.append(loss_train)
    loss_valid_hist.append(loss_valid)

    acc_train_hist.append(acc_train)
    acc_valid_hist.append(acc_valid)

    if loss_valid < best_loss_valid:
        torch.save(model, f"model.pt")
        best_loss_valid = loss_valid
        print("Model Saved!")

    print(f"Valid: Loss = {loss_valid:.4}, Acc = {acc_valid:.4}")
    print()

    epoch_counter += 1

Epoch 0: : 1875batch [01:33, 20.15batch/s, accuracy=73, loss=0.729]


Model Saved!
Valid: Loss = 0.5004, Acc = 83.99



Epoch 1: : 1875batch [01:29, 20.85batch/s, accuracy=86.1, loss=0.432]


Model Saved!
Valid: Loss = 0.4389, Acc = 85.78



Epoch 2: : 1875batch [01:30, 20.81batch/s, accuracy=86.6, loss=0.413]


...

Valid: Loss = 0.4232, Acc = 86.88



Epoch 13: : 1875batch [01:30, 20.80batch/s, accuracy=87.4, loss=0.391]


Valid: Loss = 0.3903, Acc = 87.74



Epoch 14: : 1875batch [01:31, 20.41batch/s, accuracy=87.7, loss=0.381]


Valid: Loss = 0.4104, Acc = 87.01

